<a href="https://colab.research.google.com/github/rajibmondal/Bengali.AI-Handwritten-Grapheme-Classification/blob/master/Bengali_AI_Handwritten_Grapheme_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install kaggle

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rajibmondal","key":"5bdef41fd479b5e91509300e5efde146"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# !kaggle competitions download -c digit-recognizer
!kaggle competitions download -c bengaliai-cv19

  0% 0.00/1.28M [00:00<?, ?B/s]
100% 1.28M/1.28M [00:00<00:00, 85.7MB/s]
  0% 0.00/1.32M [00:00<?, ?B/s]
100% 1.32M/1.32M [00:00<00:00, 89.1MB/s]
 99% 985M/992M [00:29<00:00, 35.7MB/s]
100% 992M/992M [00:29<00:00, 35.7MB/s]
  0% 0.00/944 [00:00<?, ?B/s]
100% 944/944 [00:00<00:00, 962kB/s]
  0% 0.00/4.75k [00:00<?, ?B/s]
100% 4.75k/4.75k [00:00<00:00, 4.91MB/s]
  0% 0.00/1.18M [00:00<?, ?B/s]
100% 1.18M/1.18M [00:00<00:00, 79.3MB/s]
  0% 0.00/4.72k [00:00<?, ?B/s]
100% 4.72k/4.72k [00:00<00:00, 4.99MB/s]
 99% 972M/986M [00:27<00:00, 30.5MB/s]
100% 986M/986M [00:27<00:00, 37.8MB/s]
  0% 0.00/16.8k [00:00<?, ?B/s]
100% 16.8k/16.8k [00:00<00:00, 17.0MB/s]
  0% 0.00/1.70k [00:00<?, ?B/s]
100% 1.70k/1.70k [00:00<00:00, 1.76MB/s]
  0% 0.00/1.30M [00:00<?, ?B/s]
100% 1.30M/1.30M [00:00<00:00, 42.9MB/s]
 98% 969M/993M [00:26<00:01, 22.0MB/s]
100% 993M/993M [00:26<00:00, 39.4MB/s]
  0% 0.00/1.25M [00:00<?, ?B/s]
100% 1.25M/1.25M [00:00<00:00, 85.3MB/s]
 99% 977M/991M [00:23<00:00, 37.9MB/s]
100%

In [4]:
ls

class_map_corrected.csv        test_image_data_2.parquet.zip
class_map.csv                  test_image_data_3.parquet.zip
kaggle.json                    train.csv.zip
sample_data/                   train_image_data_0.parquet.zip
sample_submission.csv          train_image_data_1.parquet.zip
test.csv                       train_image_data_2.parquet.zip
test_image_data_0.parquet.zip  train_image_data_3.parquet.zip
test_image_data_1.parquet.zip  train_multi_diacritics.csv


In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive')

!unzip "/content/train.csv.zip"

Archive:  /content/train.csv.zip
  inflating: train.csv               


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
from glob import glob
import time, gc
import cv2

from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns

Using TensorFlow backend.


In [0]:
train_df_ = pd.read_csv('/content/train.csv')
test_df_ = pd.read_csv('/content/test.csv')
class_map_df = pd.read_csv('/content/class_map.csv')
sample_sub_df = pd.read_csv('/content/sample_submission.csv')

In [8]:
print(f'Size of training data: {train_df_.shape}')
print(f'Size of test data: {test_df_.shape}')
print(f'Size of class map: {class_map_df.shape}')

Size of training data: (200840, 5)
Size of test data: (36, 3)
Size of class map: (186, 3)


In [0]:
# #Explore Data Analysis

# HEIGHT=240
# WIDTH=280

# def get_n(df, field, n, top=True):
#   top_graphemes = df.groupby([field]).reset_index(name='counts')['counts'].sort_values(ascending=not top)[:n]
#   top_grapheme_roots = top_graphemes.index
#   top_grapheme_counts = top_graphemes.values
#   top_graphemes = class_map_df[class_map_df['component_type'] == field].reset_index().iloc[top_grapheme_roots]
#   top_graphemes.drop(['component_type','label'], axis=1, inplace=True)
#   top_graphemes.loc[:, 'count'] = top_grapheme_counts
#   return top_graphemes

In [9]:
%%writefile /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/keras/saving/hdf5_format.py

# Copyright 2018 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
# pylint: disable=protected-access
"""Functions for saving and loading a Keras Model from HDF5 format.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import os

import numpy as np
from six.moves import zip  # pylint: disable=redefined-builtin

from tensorflow.python.framework import ops
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.saving import model_config as model_config_lib
from tensorflow.python.keras.saving import saving_utils
from tensorflow.python.keras.utils import conv_utils
from tensorflow.python.keras.utils.io_utils import ask_to_proceed_with_overwrite
from tensorflow.python.ops import variables as variables_module
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util import serialization

# pylint: disable=g-import-not-at-top
try:
  import h5py
  HDF5_OBJECT_HEADER_LIMIT = 64512
except ImportError:
  h5py = None
# pylint: enable=g-import-not-at-top


def save_model_to_hdf5(model, filepath, overwrite=True, include_optimizer=True):
  """Saves a model to a HDF5 file.
  The saved model contains:
      - the model's configuration (topology)
      - the model's weights
      - the model's optimizer's state (if any)
  Thus the saved model can be reinstantiated in
  the exact same state, without any of the code
  used for model definition or training.
  Arguments:
      model: Keras model instance to be saved.
      filepath: One of the following:
          - String, path where to save the model
          - `h5py.File` object where to save the model
      overwrite: Whether we should overwrite any existing
          model at the target location, or instead
          ask the user with a manual prompt.
      include_optimizer: If True, save optimizer's state together.
  Raises:
      ImportError: if h5py is not available.
  """

  if h5py is None:
    raise ImportError('`save_model` requires h5py.')

Writing /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/keras/saving/hdf5_format.py


FileNotFoundError: ignored

In [10]:
!pip install ../input/kaggle-efficientnet-repo/efficientnet-1.0.0-py3-none-any.whl

import os
import numpy as np
import pandas as pd
import argparse
import math
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger)
from tensorflow.keras import regularizers
from tensorflow.keras import layers as L
import efficientnet.tfkeras as efn
from tensorflow.keras.layers import Dense, Lambda, Dropout, Layer, BatchNormalization, Flatten

Processing /input/kaggle-efficientnet-repo/efficientnet-1.0.0-py3-none-any.whl
ERROR: Could not install packages due to an EnvironmentError: [Errno 2] No such file or directory: '/input/kaggle-efficientnet-repo/efficientnet-1.0.0-py3-none-any.whl'



ModuleNotFoundError: ignored